In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
import os
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score,f1_score
from tqdm import tqdm
import matplotlib as mpl
mpl.rcParams['agg.path.chunksize'] = 10000
from joblib import dump, load
import librosa.display  

from utils import data_process, visualize

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [6]:
# 读取数据
X =np.load('/home/yuanshao/EQ_Place/code/data/X_8_28.npy', allow_pickle=True)
Y =np.load('/home/yuanshao/EQ_Place/code/data/Y_8_28.npy')

pd.value_counts(Y)

0    1118
1     372
dtype: int64

In [7]:
# 统一长度

for i in range(len(X)):
    if X[i][0].shape[0]!=7201:
        X[i][0]=data_process.data_resample( X[i][0], 7201)
        X[i][1]=data_process.data_resample( X[i][1], 7201)
        X[i][2]=data_process.data_resample( X[i][2], 7201)

In [8]:
# 特征提取
feature = []
for i in tqdm(range(len(X))):
    ex, ey, ez = X[i][0], X[i][1], X[i][2]
    iqr_x = data_process.cal_IQR(ex)
    iqr_y = data_process.cal_IQR(ey)
    iqr_z = data_process.cal_IQR(ez)
    zcr_x = data_process.ZCR(ex)
    zcr_y = data_process.ZCR(ey)
    zcr_z = data_process.ZCR(ez)
    cav=data_process.CAV([ex,ey,ez])
    feature.append([iqr_x, iqr_y, iqr_z ,zcr_x, zcr_y, zcr_z, cav])
feature=np.array(feature)
feature=feature[:,:,np.newaxis,:]  
# np.save('feature_8_29',feature)

100%|██████████| 1490/1490 [00:30<00:00, 49.01it/s]


In [24]:
feature.shape

(1490, 7, 1, 36)

In [10]:
# X = np.load('./feature_8_28.npy')
# Y =np.load('/home/xxx/EQ_Place/code/data/Y_8_28.npy')
x_train, x_test, y_train, y_test = train_test_split(feature, Y,test_size=0.2, 
                                                     random_state=1)


x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train).long()

x_test = torch.from_numpy(x_test).float()
y_test = torch.from_numpy(y_test).long()
traindata =TensorDataset(x_train,y_train)
testdata = TensorDataset(x_test,y_test)
train_loader = DataLoader(traindata,batch_size = 256,shuffle=True)
test_loader = DataLoader(testdata,batch_size=256,shuffle=True)

In [10]:
x_test.shape

torch.Size([372, 7, 1, 36])

In [11]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    losst=[]
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        if(batch_idx+1)%30 == 0: 
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))#Use torch.Tensor.item() to get a Python number from a tensor containing a single value:
            losst.append(loss.item())
    return losst
            
#测试过程
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # 将一批的损失相加
            pred = output.max(1, keepdim=True)[1] # 找到概率最大的下标
            correct += pred.eq(target.view_as(pred)).sum().item()#找到正确的预测值
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return 100. * correct / len(test_loader.dataset)

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(7, 32, (1, 3), stride=1, padding=(0, 1)), nn.ReLU(),
            nn.MaxPool2d(kernel_size=(1, 2)))

        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, (1, 3), stride=1, padding=(0, 1)), nn.ReLU(),
            nn.MaxPool2d(kernel_size=(1, 2)))

        self.conv3 = nn.Sequential(
            nn.Conv2d(64, 128, (1, 3), stride=1, padding=(0, 1)), nn.ReLU(),
            nn.MaxPool2d(kernel_size=(1, 2)))

        self.fc1 =nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(256,2))
        self.fla = nn.Flatten()

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.fla(x)
        x = self.fc1(x)

        return F.log_softmax(x, dim=1)


# class CNN(nn.Module):
#     def __init__(self):
#         super(CNN, self).__init__()
#         self.conv1 = nn.Sequential(
#             nn.Conv2d(7, 32, (1, 5), stride=1, padding=(0, 1)), nn.ReLU(),
#             nn.MaxPool2d(kernel_size=(1, 3)))

#         self.conv2 = nn.Sequential(
#             nn.Conv2d(32, 64, (1, 5), stride=1, padding=(0, 1)), nn.ReLU(),
#             nn.MaxPool2d(kernel_size=(1, 3)))

#         self.conv3 = nn.Sequential(
#             nn.Conv2d(64, 128, (1, 5), stride=1, padding=(0, 1)), nn.ReLU(),
#             nn.MaxPool2d(kernel_size=(1, 3)))

#         self.fc1 =nn.Sequential(
#             nn.Linear(1536, 256),
#             nn.ReLU(),
#             nn.Dropout(p=0.5),
#             nn.Linear(256,2))
#         self.fla = nn.Flatten()

#     def forward(self, x):
#         x = self.conv1(x)
#         x = self.conv2(x)
#         x = self.conv3(x)
# #         print(x.shape)
# #         x, (h_n, c_n) = self.lstm1(x)
#         x = self.fla(x)
#         print(x.shape)
#         x = self.fc1(x)

#         return F.log_softmax(x, dim=1)

In [12]:
%%time
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
import time
L=[]
test_hist=[]
optimizer = optim.Adam(model.parameters(),lr=0.001)
for epoch in range(1, 150 + 1):
    start=time.time()
    loss=train(model, device, train_loader, optimizer, epoch)
    L.append(loss)
    print("time {:.1f} sec:".format(time.time()-start))
    acc=test(model, device, test_loader)
    test_hist.append(acc)

time 0.1 sec:

Test set: Average loss: 0.5833, Accuracy: 227/298 (76.17%)

time 0.0 sec:

Test set: Average loss: 0.5554, Accuracy: 227/298 (76.17%)

time 0.0 sec:

Test set: Average loss: 0.5401, Accuracy: 227/298 (76.17%)

time 0.0 sec:

Test set: Average loss: 0.5203, Accuracy: 227/298 (76.17%)

time 0.0 sec:

Test set: Average loss: 0.4831, Accuracy: 235/298 (78.86%)

time 0.0 sec:

Test set: Average loss: 0.4364, Accuracy: 248/298 (83.22%)

time 0.0 sec:

Test set: Average loss: 0.4199, Accuracy: 241/298 (80.87%)

time 0.0 sec:

Test set: Average loss: 0.4468, Accuracy: 239/298 (80.20%)

time 0.0 sec:

Test set: Average loss: 0.4242, Accuracy: 245/298 (82.21%)

time 0.0 sec:

Test set: Average loss: 0.4206, Accuracy: 241/298 (80.87%)

time 0.0 sec:

Test set: Average loss: 0.4101, Accuracy: 244/298 (81.88%)

time 0.0 sec:

Test set: Average loss: 0.4113, Accuracy: 242/298 (81.21%)

time 0.0 sec:

Test set: Average loss: 0.4073, Accuracy: 243/298 (81.54%)

time 0.0 sec:

Test set: 

time 0.0 sec:

Test set: Average loss: 0.1118, Accuracy: 289/298 (96.98%)

time 0.0 sec:

Test set: Average loss: 0.1101, Accuracy: 289/298 (96.98%)

time 0.0 sec:

Test set: Average loss: 0.1028, Accuracy: 289/298 (96.98%)

time 0.0 sec:

Test set: Average loss: 0.0925, Accuracy: 288/298 (96.64%)

time 0.0 sec:

Test set: Average loss: 0.1079, Accuracy: 287/298 (96.31%)

time 0.0 sec:

Test set: Average loss: 0.0932, Accuracy: 288/298 (96.64%)

time 0.0 sec:

Test set: Average loss: 0.1944, Accuracy: 283/298 (94.97%)

time 0.0 sec:

Test set: Average loss: 0.0971, Accuracy: 286/298 (95.97%)

time 0.0 sec:

Test set: Average loss: 0.1195, Accuracy: 283/298 (94.97%)

time 0.0 sec:

Test set: Average loss: 0.1014, Accuracy: 289/298 (96.98%)

time 0.0 sec:

Test set: Average loss: 0.1106, Accuracy: 289/298 (96.98%)

time 0.0 sec:

Test set: Average loss: 0.1279, Accuracy: 284/298 (95.30%)

time 0.0 sec:

Test set: Average loss: 0.1173, Accuracy: 291/298 (97.65%)

time 0.0 sec:

Test set: 

In [13]:
x_test=x_test.to(device)

y_test=y_test.to(device)

pred=model(x_test)
pred = pred.max(1, keepdim=True)[1]

correct = pred.eq(y_test.view_as(pred)).sum().item()

pred=pred.cpu().numpy().squeeze()

In [14]:
confusion_matrix(pred, y_test.cpu())

array([[221,   2],
       [  6,  69]])

In [15]:
from sklearn.metrics import classification_report
print(classification_report(pred, y_test.cpu()))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98       223
           1       0.97      0.92      0.95        75

    accuracy                           0.97       298
   macro avg       0.97      0.96      0.96       298
weighted avg       0.97      0.97      0.97       298



In [16]:
f1_score(pred, y_test.cpu())

0.9452054794520549

In [17]:
from sklearn.metrics import roc_auc_score
roc_auc_score(pred, y_test.cpu())

0.9555156950672645

In [19]:
x_test = np.load('./data/feature_8_29.npy')
y_test =np.load('/home/yuanshao/EQ_Place/code/data/Y_8_29.npy')

In [20]:
x_test = torch.from_numpy(x_test).float()
y_test = torch.from_numpy(y_test).long()
traindata =TensorDataset(x_train,y_train)
testdata = TensorDataset(x_test,y_test)
train_loader = DataLoader(traindata,batch_size = 256,shuffle=True)
test_loader = DataLoader(testdata,batch_size=256,shuffle=True)

In [21]:
len(x_test)

1859

In [23]:
x_test.shape

torch.Size([1859, 7, 1, 360])

In [ ]:
pred=model(x_test)
pred = pred.max(1, keepdim=True)[1]
correct = pred.eq(y_test.view_as(pred)).sum().item()
pred=pred.cpu().numpy().squeeze()